In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
import matplotlib.pylab as plt
plt.style.use('bmh')
from sklearn.linear_model import Ridge

In [2]:
df_test = pd.read_csv('./test.csv')
df_ex = pd.read_csv('./sample_submission.csv')
df_train = pd.read_csv('./sales_train.csv')
df_shop = pd.read_csv('./shops.csv')
df_items = pd.read_csv('./items.csv')
df_cat = pd.read_csv('./item_categories.csv')

In [3]:
# Get item sells by month
grouped = df_train.groupby(['item_id', 'shop_id', 'date_block_num'])
month_stat = grouped['item_cnt_day'].sum()

In [4]:
# Create train table with "item_cnt_month"
X = np.array([*month_stat.index.values])
Y = np.array(month_stat.values)
df_train_m = pd.DataFrame({'date_block_num':X[:,2], 'item_id':X[:,0], 'shop_id':X[:,1], 'item_cnt_m':Y})
df_train_m = pd.merge(df_train_m, df_items[['item_id', 'item_category_id']], on='item_id')
df_train_m = df_train_m.sort_values(by='date_block_num')
df_train_m.head()

,date_block_num,item_cnt_m,item_id,shop_id,item_category_id
747775,0,1.0,9607,46,40
830176,0,1.0,10904,27,55
321175,0,1.0,4244,2,23
321172,0,1.0,4244,0,23
1292935,0,1.0,17114,44,40


In [5]:
Y = df_train_m['item_cnt_m']
X0 = df_train_m[['item_id', 'shop_id', 'item_category_id', 'date_block_num']]
X0.head()

,item_id,shop_id,item_category_id,date_block_num
747775,9607,46,40,0
830176,10904,27,55,0
321175,4244,2,23,0
321172,4244,0,23,0
1292935,17114,44,40,0


In [6]:
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.preprocessing import OneHotEncoder

In [7]:
rgr_ridge = Ridge()
dtr = DecisionTreeRegressor()
ada = AdaBoostRegressor()
bag = BaggingRegressor(verbose=2)
random_forest = RandomForestRegressor(verbose=2)
gboost = GradientBoostingRegressor(verbose=2)

In [9]:
gboost.fit(X0, Y)

      Iter       Train Loss   Remaining Time 
         1          71.4625            2.27m
         2          68.7375            2.23m
         3          66.5228            2.32m
         4          64.7057            2.34m
         5          63.2268            2.35m
         6          62.0068            2.39m
         7          60.9788            2.45m
         8          60.1401            2.40m
         9          59.4255            2.38m
        10          58.8387            2.37m
        11          58.3321            2.35m
        12          57.9124            2.32m
        13          57.5440            2.28m
        14          57.3026            2.25m
        15          57.2597            2.23m
        16          57.0697            2.22m
        17          57.0339            2.21m
        18          56.9730            2.17m
        19          56.8837            2.16m
        20          56.8573            2.12m
        21          56.8344            2.08m
        2

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=100,
             presort='auto', random_state=None, subsample=1.0, verbose=2,
             warm_start=False)

### Test data

In [10]:
df_ex.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [11]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [12]:
df_test_a = pd.merge(df_test, df_items[['item_id', 'item_category_id']], on='item_id')
df_test_a = df_test_a.sort_values(by='ID')
df_test_a['date_block_num'] = [34]*len(df_test_a)
X0_test = df_test_a[['item_id', 'shop_id', 'item_category_id', 'date_block_num']]

In [19]:
Y_pred = gboost.predict(X0_test)
df_ex['item_cnt_month'] = Y_pred
df_ex = df_ex.set_index('ID')

In [21]:
df_ex.to_csv('pred.csv')